1. PDF → Metin al ve node çıkar (llm ile otomatik) nodeları etiketle[[]] ile

In [1]:
text = "Yapay zeka sistemlerinin etik yönleri çok önemlidir. Regülasyon konusu da bununla ilgilidir."
concepts = ["Yapay zeka", "etik", "Regülasyon"]
import re

def link_concepts(text, concepts):
    for concept in sorted(concepts, key=len, reverse=True):
        text = re.sub(rf"\b({re.escape(concept)})\b", r"[[\1]]", text)
    return text

linked_text = link_concepts(text, concepts)
print(linked_text)


[[Yapay zeka]] sistemlerinin [[etik]] yönleri çok önemlidir. [[Regülasyon]] konusu da bununla ilgilidir.


 Node ve Edge çıkart + bağlamları tut

In [2]:
def extract_edges_and_contexts(linked_text, main_node="Text_1"):
    pattern = r'(.{0,50})\[\[(.*?)\]\](.{0,50})'
    matches = re.findall(pattern, linked_text)

    edges, contexts, nodes = [], [], set()
    for before, concept, after in matches:
        context = f"{before.strip()} {concept} {after.strip()}"
        edges.append({"from": main_node, "to": concept})
        contexts.append(context)
        nodes.add(concept) 

    nodes.add(main_node)
    return list(nodes), edges, contexts
nodes, edges, contexts = extract_edges_and_contexts(linked_text)

TF-IDF ile edge etiketleri (bağlam kelimesi) ya da llm e etik’ arasında nasıl bir ilişki vardır? Bunu bir kelime veya kısa bir açıklama ile özetle.”

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

def label_edges_with_tfidf(contexts):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(contexts)
    labels = []
    for i in range(len(contexts)):
        row = X[i].toarray()[0]
        top_index = row.argmax()
        labels.append(vectorizer.get_feature_names_out()[top_index])
    return labels

labels = label_edges_with_tfidf(contexts)


graph.json

In [4]:
def build_graph_json(nodes, edges, labels):
    for i, label in enumerate(labels):
        edges[i]["label"] = label

    node_objs = [{"id": n, "label": n} for n in nodes]
    return {
        "nodes": node_objs,
        "edges": edges
    }

graph = build_graph_json(nodes, edges, labels)

import json
with open("graph.json", "w", encoding="utf-8") as f:
    json.dump(graph, f, ensure_ascii=False, indent=2)


In [5]:
print(json.dumps(graph, indent=2, ensure_ascii=False))


{
  "nodes": [
    {
      "id": "etik",
      "label": "etik"
    },
    {
      "id": "Text_1",
      "label": "Text_1"
    }
  ],
  "edges": [
    {
      "from": "Text_1",
      "to": "etik",
      "label": "da"
    }
  ]
}


Cytoscape.js ile Graph Görselleştirme

In [10]:
!pip install pyvis


  Obtaining dependency information for pyvis from https://files.pythonhosted.org/packages/ab/4b/e37e4e5d5ee1179694917b445768bdbfb084f5a59ecd38089d3413d4c70f/pyvis-0.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpickle>=1.4.1 from https://files.pythonhosted.org/packages/c1/73/04df8a6fa66d43a9fd45c30f283cc4afff17da671886e451d52af60bdc7e/jsonpickle-4.1.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 165.8 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 782.3 kB/s eta 0:00:000:00:01


In [6]:
from pyvis.network import Network

# Mini graph oluştur
net = Network(notebook=True, height="400px", width="80%")

# Node'ları ekle
for node in nodes:
    net.add_node(node, label=node)

# Edge'leri ekle
for i, edge in enumerate(edges):
    label = labels[i] if i < len(labels) else ""
    net.add_edge(edge['from'], edge['to'], label=label)

# Görselleştir
net.show("pyvis_graph.html")


pyvis_graph.html


In [7]:
from IPython.display import IFrame
IFrame("pyvis_graph.html", width="100%", height="450px")


In [8]:
from pptx import Presentation
from pptx.util import Inches, Pt

# Yeni sunum oluştur
prs = Presentation()

# Slayt ekleme fonksiyonu
def add_slide(title, content):
    slide_layout = prs.slide_layouts[1]  # Title and Content
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    slide.placeholders[1].text = content

# Slayt içerikleri
slides = [
    ("Proje Amacı",
     "• PDF veya düz metinlerden otomatik kavram çıkarımı\n"
     "• Kavramlar arasındaki ilişkilerin bağlamlı şekilde bulunması\n"
     "• Elde edilen yapının graf olarak görselleştirilmesi"),

    ("Sistem Akışı",
     "PDF / Metin\n"
     "   ↓\n"
     "Kavram Çıkarımı (LLM / Liste)\n"
     "   ↓\n"
     "Metin İçinde Etiketleme [[concept]]\n"
     "   ↓\n"
     "Node & Edge Çıkarımı\n"
     "   ↓\n"
     "Bağlam Yakalama\n"
     "   ↓\n"
     "Edge Etiketleme (TF-IDF / LLM)\n"
     "   ↓\n"
     "Graph JSON\n"
     "   ↓\n"
     "Görselleştirme (PyVis / Cytoscape.js)"),

    ("Node’lar",
     "Örnek metin:\n"
     "Yapay zeka sistemlerinin etik yönleri çok önemlidir.\n"
     "Regülasyon konusu da bununla ilgilidir.\n\n"
     "Kavramlar: Yapay zeka, Etik, Regülasyon\n"
     "Metin içi etiketleme:\n"
     "[[Yapay zeka]] sistemlerinin [[etik]] yönleri...\n"
     "[[Regülasyon]] konusu da ..."),

    ("Graph Yapısı",
     "Node: Kavramlar ve ana metin (Text_1)\n"
     "Edge: Kavramlar → Metin\n"
     "Örnek:\n"
     "Text_1 ──> Yapay zeka\n"
     "Text_1 ──> Etik\n"
     "Text_1 ──> Regülasyon"),

    ("Bağlam ile Etiketleme",
     "Her edge için ±50 karakter bağlam yakalanır\n"
     "TF-IDF ile bağlamın temsil eden kelimesi edge label olur\n"
     "Örnek:\n"
     "Edge: Text_1 → Etik\n"
     "Label: \"önem\""),

    ("Sonuçlar ve Görselleştirme",
     "JSON formatı:\n"
     '{ "nodes": [{"id":"etik","label":"etik"},{"id":"Text_1","label":"Text_1"}],\n'
     '  "edges": [{"from":"Text_1","to":"etik","label":"önem"}] }\n'
     "PyVis ile görselleştirme\n"
     "Network graph: Nodes → kavramlar, Edges → ilişkiler"),

    ("Avantajlar",
     "• Metni makinenin anlayabileceği bilgi ağına çevirir\n"
     "• TF-IDF ve LLM ile bağlamlı ilişkiler çıkarır\n"
     "• Multi-document graph’a ölçeklenebilir\n"
     "• Frontend ile interaktif görselleştirilebilir"),

    ("Özet",
     "• PDF/metin → kavram → bağlamlı edge → graph\n"
     "• TF-IDF ve LLM hibrit yaklaşımı\n"
     "• Görselleştirme ve analiz için hazır yapı")
]

# Slaytları ekle
for title, content in slides:
    add_slide(title, content)

# Sunumu kaydet
prs.save("Knowledge_Graph_Projesi_Sunum.pptx")
print("Sunum kaydedildi: Knowledge_Graph_Projesi_Sunum.pptx")


Sunum kaydedildi: Knowledge_Graph_Projesi_Sunum.pptx
